In [58]:
from pyspark.sql import SparkSession
from pyspark.sql import *
from pyspark.sql.types import DoubleType,IntegerType
import pyspark.sql.functions as F

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .getOrCreate()

df  = spark.read.csv("WorldCupMatches.csv", header=True)

In [59]:
df.show()

+----+--------------------+-----------------+--------------------+-----------+--------------+---------------+---------------+--------------+--------------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+-------+-------+------------------+------------------+
|Year|            Datetime|            Stage|             Stadium|       City|Home Team Name|Home Team Goals|Away Team Goals|Away Team Name|      Win conditions|Attendance|Half-time Home Goals|Half-time Away Goals|             Referee|         Assistant 1|         Assistant 2|RoundID|MatchID|Home Team Initials|Away Team Initials|
+----+--------------------+-----------------+--------------------+-----------+--------------+---------------+---------------+--------------+--------------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+-------+-------+------------------+------------------+
|193

In [60]:
df.printSchema

<bound method DataFrame.printSchema of DataFrame[Year: string, Datetime: string, Stage: string, Stadium: string, City: string, Home Team Name: string, Home Team Goals: string, Away Team Goals: string, Away Team Name: string, Win conditions: string, Attendance: string, Half-time Home Goals: string, Half-time Away Goals: string, Referee: string, Assistant 1: string, Assistant 2: string, RoundID: string, MatchID: string, Home Team Initials: string, Away Team Initials: string]>

In [61]:
df  = df.withColumnRenamed('Home Team Name', 'Home_Team')
df  = df.withColumnRenamed('Away Team Name', 'Away_Team')
df  = df.withColumnRenamed('Away Team Goals', 'Away_Team_Goals')
df  = df.withColumnRenamed('Home Team Goals', 'Home_Team_Goals')
df  = df.withColumn('Away_Team_Goals', df['Away_Team_Goals'].cast(DoubleType()))
df  = df.withColumn('Home_Team_Goals', df['Home_Team_Goals'].cast(DoubleType()))
df  = df.withColumn('Attendance', df['Attendance'].cast(IntegerType()))

In [62]:
df.printSchema

<bound method DataFrame.printSchema of DataFrame[Year: string, Datetime: string, Stage: string, Stadium: string, City: string, Home_Team: string, Home_Team_Goals: double, Away_Team_Goals: double, Away_Team: string, Win conditions: string, Attendance: int, Half-time Home Goals: string, Half-time Away Goals: string, Referee: string, Assistant 1: string, Assistant 2: string, RoundID: string, MatchID: string, Home Team Initials: string, Away Team Initials: string]>

In [63]:
df.show()

+----+--------------------+-----------------+--------------------+-----------+----------+---------------+---------------+----------+--------------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+-------+-------+------------------+------------------+
|Year|            Datetime|            Stage|             Stadium|       City| Home_Team|Home_Team_Goals|Away_Team_Goals| Away_Team|      Win conditions|Attendance|Half-time Home Goals|Half-time Away Goals|             Referee|         Assistant 1|         Assistant 2|RoundID|MatchID|Home Team Initials|Away Team Initials|
+----+--------------------+-----------------+--------------------+-----------+----------+---------------+---------------+----------+--------------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+-------+-------+------------------+------------------+
|1930|13 Jul 1930 - 15:00 | 

In [73]:
# Query 1 :  Find the number of games played in each Year
df.filter(df.Year!=" ")\
.select(df.Year)\
.groupBy(df.Year)\
.count()\
.orderBy(df.Year).show()

+----+-----+
|Year|count|
+----+-----+
|1930|   18|
|1934|   17|
|1938|   18|
|1950|   22|
|1954|   26|
|1958|   35|
|1962|   32|
|1966|   32|
|1970|   32|
|1974|   38|
|1978|   38|
|1982|   52|
|1986|   52|
|1990|   52|
|1994|   52|
|1998|   64|
|2002|   64|
|2006|   64|
|2010|   64|
|2014|   80|
+----+-----+



In [74]:
# Query 2: Finding the distinct statium in the world cup dataset
df.select(df.Stadium).distinct().show()

+--------------------+
|             Stadium|
+--------------------+
|          Cuauhtemoc|
|      Parque Central|
|       Idrottsparken|
|         Waldstadion|
|              Friuli|
|       Jose Zorrilla|
|Old Trafford Stadium|
|           San Mames|
|      Miyagi Stadium|
|FIFA World Cup St...|
|Royal Bafokeng Sp...|
|       Nuevo Estadio|
|      Arena Amazonia|
|Nou Camp - Estadi...|
|   Santiago Bernabeu|
| Osaka Nagai Stadium|
|Estadio Jos� Mar�...|
|Ramon Sanchez Piz...|
|     Renato Dall Ara|
|  Pontiac Silverdome|
+--------------------+
only showing top 20 rows



In [75]:
# Query 3: Find the top 10 maximum people attended stadiums
df.select(df.Stadium,df.Attendance)\
.groupBy(df.Stadium)\
.sum()\
.orderBy('sum(Attendance)',ascending=False).show()

+--------------------+---------------+
|             Stadium|sum(Attendance)|
+--------------------+---------------+
|      Estadio Azteca|        1917550|
|      Olympiastadion|         815300|
|Maracan� - Est�di...|         813541|
|     Wembley Stadium|         778034|
| Estadio do Maracana|         741971|
|           Rose Bowl|         740807|
|             Jalisco|         722456|
|     Stade de France|         704000|
|    Estadio Nacional|         682685|
| Soccer City Stadium|         670859|
|            Nacional|         663771|
|El Monumental - E...|         625389|
|      Giants Stadium|         515828|
|   Cape Town Stadium|         507340|
|  Arena de Sao Paulo|         502115|
|    Stanford Stadium|         490424|
|    Estadio Castelao|         476055|
|  Estadio Centenario|         471881|
|    Estadio Mineirao|         461205|
|     Stadio Olimpico|         440238|
+--------------------+---------------+
only showing top 20 rows



In [82]:
# Query 4: Finding the Win Conditions and their distributions
df=df.withColumnRenamed('Win conditions','Win_conditions')
df.filter(df.Win_conditions!=' ').select(df.Win_conditions)\
.groupBy(df.Win_conditions)\
.count()\
.orderBy('count',ascending=False)\
.show(truncate=False)

+-------------------------------------+-----+
|Win_conditions                       |count|
+-------------------------------------+-----+
|Italy win after extra time           |5    |
|Argentina win after extra time       |4    |
|Germany win after extra time         |4    |
|Belgium win after extra time         |3    |
|Brazil win on penalties (3 - 2)      |3    |
|England win after extra time         |3    |
|Win on Golden Goal                   |3    |
|Argentina win on penalties (2 - 4)   |2    |
|France win after extra time          |2    |
|Netherlands win on penalties (4 - 3) |2    |
|France win on penalties (3 - 4)      |2    |
|Costa Rica win on penalties (5 - 3)  |2    |
|Ukraine win on penalties (0 - 3)     |1    |
|Ghana win after extra time           |1    |
|Austria win after extra time         |1    |
|Sweden win on penalties (4 - 5)      |1    |
|Uruguay win after extra time         |1    |
|Argentina win on penalties (4 - 3)   |1    |
|Italy win on penalties (5 - 3)   

In [86]:
# Query 5:Sum of home goals scored and grouped by home team country 
df.filter(df.Home_Team!=' ')\
.groupBy('Home_Team')\
.sum('Home_Team_Goals')\
.orderBy('Home_Team')\
.show(truncate=False)

+-----------------------------+--------------------+
|Home_Team                    |sum(Home_Team_Goals)|
+-----------------------------+--------------------+
|"rn"">Bosnia and Herzegovina"|3.0                 |
|"rn"">Republic of Ireland"   |2.0                 |
|"rn"">Serbia and Montenegro" |0.0                 |
|"rn"">Trinidad and Tobago"   |0.0                 |
|"rn"">United Arab Emirates"  |0.0                 |
|Algeria                      |5.0                 |
|Angola                       |0.0                 |
|Argentina                    |111.0               |
|Australia                    |7.0                 |
|Austria                      |31.0                |
|Belgium                      |27.0                |
|Bolivia                      |1.0                 |
|Brazil                       |180.0               |
|Bulgaria                     |11.0                |
|Cameroon                     |11.0                |
|Canada                       |0.0            

In [93]:
# Query 6: Sum of home goals scored in each phase stage
df.filter(df.Stage!="")\
.groupBy('Stage')\
.sum('Home_Team_Goals')\
.orderBy('Stage')\
.show(truncate=False)

+------------------------+--------------------+
|Stage                   |sum(Home_Team_Goals)|
+------------------------+--------------------+
|Final                   |43.0                |
|First round             |30.0                |
|Group 1                 |129.0               |
|Group 2                 |142.0               |
|Group 3                 |116.0               |
|Group 4                 |127.0               |
|Group 5                 |5.0                 |
|Group 6                 |43.0                |
|Group A                 |77.0                |
|Group B                 |100.0               |
|Group C                 |77.0                |
|Group D                 |73.0                |
|Group E                 |65.0                |
|Group F                 |54.0                |
|Group G                 |42.0                |
|Group H                 |39.0                |
|Match for third place   |38.0                |
|Play-off for third place|0.0           

In [94]:
# Query 7: Display the max Home goals scored year wise 
df.groupBy('Year').max('Home_Team_Goals').show()

+----+--------------------+
|Year|max(Home_Team_Goals)|
+----+--------------------+
|1958|                 7.0|
|1938|                 8.0|
|2014|                 4.0|
|1982|                10.0|
|1962|                 6.0|
|1954|                 9.0|
|null|                null|
|1930|                 6.0|
|1978|                 6.0|
|1974|                 9.0|
|2002|                 8.0|
|2006|                 6.0|
|1966|                 5.0|
|1994|                 6.0|
|1950|                 8.0|
|1934|                 7.0|
|1986|                 6.0|
|1998|                 6.0|
|2010|                 7.0|
|1990|                 5.0|
+----+--------------------+
only showing top 20 rows



In [95]:
# Query 8: pairing stage wise matches with Away_Team
df.crosstab('Stage', 'Away_Team').show()

+--------------------+-----------------------------+--------------------------+----------------------------+--------------------------+---------------------------+-------+------+---------+---------+-------+-------+-------+------+--------+--------+------+-----+--------+--------+----------+-------+----+--------------+--------------+-------------+-------+-----------------+-------+-----+-----------+-------+------+---------+-------+----------+-----+------+-----+--------+-------+-------+----+----+------+-----+-------+-----+---------+--------------+------+------+-------+-----------+-----------+-------+----------------+------+--------+----+------+--------+-------+------+------------+--------+-------+------+--------+--------+------------+------------+-----+------+-----------+----+-------+------+---+-------+-------+-----+----------+-----+----+
|     Stage_Away_Team|"rn"">Bosnia and Herzegovina"|"rn"">Republic of Ireland"|"rn"">Serbia and Montenegro"|"rn"">Trinidad and Tobago"|"rn"">United Arab E

In [96]:
# Query 9: Count the total no. of matches played in 2010 
print(df.filter(df.Year.like("2010")).count())

64


In [99]:
# Query 10: Describing mean, max, min of the Home_Team_Goals
df.describe(['Year','Home_Team_Goals','Away_Team_Goals']).show()

+-------+------------------+------------------+------------------+
|summary|              Year|   Home_Team_Goals|   Away_Team_Goals|
+-------+------------------+------------------+------------------+
|  count|               852|               852|               852|
|   mean|1985.0892018779343|1.8110328638497653| 1.022300469483568|
| stddev|22.448824702021138|1.6102551385229658|1.0875733783096067|
|    min|              1930|               0.0|               0.0|
|    max|              2014|              10.0|               7.0|
+-------+------------------+------------------+------------------+



In [ ]:
!pwd